### Contents

[Imports](#Imports)

[bottom](#bottom)

### Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pprint import pprint
import sys 
import json 
import itertools 
from tqdm.auto import tqdm
from datetime import datetime, date, timedelta
from load_dotenv import load_dotenv
load_dotenv() # OPENAI_API_KEY

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from IPython.display import display, clear_output

/Users/antonandreytsev/llm_zoomcamp_venv3.9.7/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
matplotlib.rcParams['figure.figsize'] = (7, 7)
sns.set_style('whitegrid')

---

In [4]:
import openai
from openai import OpenAI

In [5]:
# os.environ['OPENAI_API_KEY'] = 'xxxx'

In [6]:
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)
client

In [7]:
# q = 'What is the capital of Russia?'
q = "What is the purpose of human life?"
response = client.chat.completions.create(
    model="phi3",
    messages=[{"role": "user", "content": q}]
)
response_content = response.choices[0].message.content
print(response_content)

 The question of the purpose of human life has intrigued philosophers, theologians, scientists, and thinkers for centuries. It's a deeply personal and subjective inquiry that can vary greatly from person to person based on their beliefs, values, experiences, and aspirations. However, I can outline several perspectives through which one might find meaning or purpose in life:

1. Philosophical perspective: Many philosophers have proposed various reasons for the purpose of human existence, such as seeking happiness, pursuing knowledge, achieving virtue, contributing to society's progress, and creating personal legacies. Existentialists like Jean-Paul Sartre or Albert Camus explored the idea that humans define their own purpose through free will and individual choice.

2. Religious/spiritual perspective: Different religions offer various interpretations of life's purpose, such as fulfilling a divine plan, achieving spiritual enlightenment, fostering love and compassion towards others, or l

In [8]:
q = "What is the purpose of human life?"
response = client.chat.completions.create(
    model="gemma:2b",
    messages=[{"role": "user", "content": q}]
)
response_content = response.choices[0].message.content
print(response_content)

The purpose of human life is a profound and multifaceted question that has captivated philosophers, theologians, and individuals throughout history. There is no single, universally accepted answer, as the meaning of life is deeply personal and subjective.

**Common perspectives on the purpose of human life include:**

**1. Biological purpose:**

* To perpetuate the human species and ensure the continuation of life on Earth.
* To fulfill the biological instincts and desires that drive us, such as the need for food, reproduction, and survival.

**2. Existentialist perspective:**

* To explore and discover our own existence and meaning in a vast and indifferent universe.
* To challenge the meaninglessness of existence and find our own purpose in the process of self-reflection.

**3. Religious perspectives:**

* To fulfill divine plans and achieve spiritual enlightenment.
* To achieve forgiveness, redemption, and a deeper connection with a higher power.

**4. Philosophical perspectives:**


In [7]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
print(len(documents))
print(documents[:2])

948
[{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.", 'section': 'General course-related questions', 'question': 'Course - When will the course start?', 'course': 'data-engineering-zoomcamp'}, {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites', 'section': 'General course-related questions', 'question': 'Course - What are the prerequisites for this course?', 'course': 'data-engineering-zoomcamp'}]


### Q2. Indexing the data
Index the data in the same way as was shown in the course videos. Make the course field a keyword and the rest should be text.

Don't forget to install the ElasticSearch client for Python:

In [21]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch("http://localhost:9200")
es_client.info()

ObjectApiResponse({'name': '5cca4a99c596', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'qGWRjdWnRGKNbgCTgHPxAQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [23]:
es_client.indices.create(
    index='course-questions',
    body={
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 0
        },
        "mappings": {
            "properties": {
                "text": {"type": "text"},
                "section": {"type": "text"},
                "question": {"type": "text"},
                "course": {"type": "keyword"} 
            }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [26]:
for doc in tqdm(documents):
    es_client.index(index='course-questions', document=doc)

100%|██████████| 948/948 [00:02<00:00, 431.11it/s]


### Q3. Searching
Now let's search in our index.

We will execute a query "How do I execute a command in a running docker container?".

Use only question and text fields and give question a boost of 4, and use "type": "best_fields".

What's the score for the top ranking result?

In [72]:
query = "How do I execute a command in a running docker container?"

es_search_template = {
    "size": 1,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": [
                        "question^4", 
                        "text", 
                        # "section"
                    ],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

response = es_client.search(
    index='course-questions', 
    body=es_search_template,
)
response

ObjectApiResponse({'took': 21, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 390, 'relation': 'eq'}, 'max_score': 75.54128, 'hits': [{'_index': 'course-questions', '_id': 'CzgBUZABPf-Lxy7JT5Q3', '_score': 75.54128, '_source': {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork name (docker network)\npostgres related variables for pgcli\nHostname\nUsername\nPort\nDatabase name\n$ docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1\n175dd47cda07:/# pgcli -h pg-database -U root -p 5432 -d ny_taxi\nPassword for root:\nServer: PostgreSQL 16.1 (Debian 16.1-1.pgdg120+1)\nVersion: 4.0.1\nHome: http://pgcli.com\nroot@pg-database:ny_taxi> \\dt\n+--------+------------------+-------+-------+\n| Schema | Name             | Type  | Owner |\n

In [75]:
response['hits']['hits']

[{'_index': 'course-questions',
  '_id': 'CzgBUZABPf-Lxy7JT5Q3',
  '_score': 75.54128,
  '_source': {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork name (docker network)\npostgres related variables for pgcli\nHostname\nUsername\nPort\nDatabase name\n$ docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1\n175dd47cda07:/# pgcli -h pg-database -U root -p 5432 -d ny_taxi\nPassword for root:\nServer: PostgreSQL 16.1 (Debian 16.1-1.pgdg120+1)\nVersion: 4.0.1\nHome: http://pgcli.com\nroot@pg-database:ny_taxi> \\dt\n+--------+------------------+-------+-------+\n| Schema | Name             | Type  | Owner |\n|--------+------------------+-------+-------|\n| public | yellow_taxi_data | table | root  |\n+--------+------------------+-------+-------+\nSELECT 1\nTime: 0.009s\nroot@pg-database:ny_taxi>',
   '

In [64]:

print(query, end='\n'+'-'*70+'\n'*2)
for i, metadata in enumerate(response['hits']['hits']):
    print(i)
    print(metadata['_source']['text'], end='\n'*2) 

How do I execute a command in a running docker container?
----------------------------------------------------------------------

0
In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.
Below the usage with values used in the videos of the course for:
network name (docker network)
postgres related variables for pgcli
Hostname
Username
Port
Database name
$ docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1
175dd47cda07:/# pgcli -h pg-database -U root -p 5432 -d ny_taxi
Password for root:
Server: PostgreSQL 16.1 (Debian 16.1-1.pgdg120+1)
Version: 4.0.1
Home: http://pgcli.com
root@pg-database:ny_taxi> \dt
+--------+------------------+-------+-------+
| Schema | Name             | Type  | Owner |
|--------+------------------+-------+-------|
| public | yellow_taxi_data | table | root  |
+--------+------------------+-------+-------+
SELECT 1
Time: 0.009s
root@pg-database:ny_taxi>

1
You 

### Q4. Filtering
Now let's only limit the questions to machine-learning-zoomcamp.

Return 3 results. What's the 3rd question returned by the search engine?

- How do I debug a docker container?
- How do I copy files from a different folder into docker container’s working directory?
- How do Lambda container images work?
- How can I annotate a graph?

In [78]:
query = "How do I execute a command in a running docker container?"

es_search_template = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": [
                        "question^4", 
                        "text", 
                        # "section"
                    ],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

response = es_client.search(
    index='course-questions', 
    body=es_search_template,
)
response

ObjectApiResponse({'took': 22, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 345, 'relation': 'eq'}, 'max_score': 84.050095, 'hits': [{'_index': 'course-questions', '_id': '_TgBUZABPf-Lxy7JU5V3', '_score': 84.050095, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'HDgBUZABPf-Lxy7JU5a5', '_score': 51.04628, '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo 

In [79]:
response['hits']['hits']

[{'_index': 'course-questions',
  '_id': '_TgBUZABPf-Lxy7JU5V3',
  '_score': 84.050095,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'HDgBUZABPf-Lxy7JU5a5',
  '_score': 51.04628,
  '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_

### Q5. Building a prompt
Now we're ready to build a prompt to send to an LLM.

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (\n\n)

```python
context_template = """
Q: {question}
A: {text}
""".strip()
```
Now use the context you just created along with the "How do I execute a command in a running docker container?" question to construct a prompt using the template below:

```python
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
```
What's the length of the resulting prompt? (use the len function)

In [89]:
context = '\n\n'.join(
f"""
Q: {metadata['_source']['question']}
A: {metadata['_source']['text']}
""".strip()
    for i, metadata in enumerate(response['hits']['hits'], start=1) if i <= 3
)
print(context)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [90]:
question = "How do I execute a command in a running docker container?"

prompt = f"""
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

In [91]:
print(len(prompt))

1462


In [102]:
completion = openai.chat.completions.create(
    model='gpt-4o',
    messages=[
        {
            'role': 'system',
            'content': 'You are a precise assistant. Provide answer in parsable JSON format.',
        },
        {
            'role': 'user',
            'content': prompt,
        },
    ],
    response_format={ "type": "json_object" },
    temperature=0,
  )

llm_response = json.loads(completion.choices[0].message.content)
pprint(llm_response)

{'answer': 'To execute a command in a running Docker container, you can use '
           'the `docker exec` command. First, find the container ID using '
           '`docker ps`, then execute the command using `docker exec -it '
           '<container-id> <command>`. For example, to start a bash session, '
           'you would use: `docker exec -it <container-id> bash`.'}


In [99]:
# response without prompt
completion = openai.chat.completions.create(
    model='gpt-4o',
    messages=[
        {
            'role': 'system',
            'content': 'You are a precise assistant. Provide answer in parsable JSON format.',
        },
        {
            'role': 'user',
            'content': question,
        },
    ],
    response_format={ "type": "json_object" },
    temperature=0,
  )

llm_response_without_context = json.loads(completion.choices[0].message.content)
pprint(llm_response_without_context)

{'instructions': [{'command': 'docker ps',
                   'description': 'Identify the container ID or name of the '
                                  'running Docker container.',
                   'step': 1},
                  {'command_template': 'docker exec [OPTIONS] CONTAINER '
                                       'COMMAND [ARG...]',
                   'description': 'Execute the desired command in the running '
                                  'Docker container using the following '
                                  'syntax.',
                   'example': 'docker exec -it my_container /bin/bash',
                   'step': 2}]}


### Q6

In [103]:
import tiktoken 
encoding = tiktoken.encoding_for_model("gpt-4o")

In [106]:
len(encoding.encode(prompt))

322

### bottom